In [13]:
!pip install librosa

In [14]:
import numpy as np
from numpy.polynomial import polynomial as p
from sympy import *
from sympy.core.numbers import Zero
import math
import librosa as lr
import scipy.io.wavfile as wavf
import os

In [65]:
class Filter:

    def __init__(self, weights, indexes):

        if len(weights) != len(indexes):
            raise Exception("Lengths of values and indices can't be different!")

        self.weights = weights
        self.indexes = indexes
        self.base = indexes[0]
        self.end = indexes[-1]

    def __str__(self):

        return str((self.weights, self.indexes))

    def __add__(self, other):

        new_weights = []
        new_indexes = list(range(min(self.indexes[0], other.indexes[0]), 
                                max(self.indexes[-1], other.indexes[-1]) + 1))

        for index in range(len(new_indexes)):
            new_weights.append(self[index] + other[index])

        return Filter(new_weights, new_indexes)

    def __mul__(self, other):

        if other == upsampling:
            return upsampling(self)

        elif other == downsampling:
            return downsampling(self)

        new_weights = []
        new_indexes = list(range(self.indexes[0] + other.indexes[0],
                         self.indexes[-1] + other.indexes[-1] + 1))

        for n in new_indexes:
            value = 0
            for k in range(max(n - other.end, self.base), min(n - other.base, self.end)+1):
                value += self[k] * other[n - k]
            new_weights.append(value)

        return Filter(new_weights, new_indexes)
    
    def __getitem__(self, key):
        
        if type(key) != int:
            raise Exception("index should have an int type")
            
        if self.base <= key <= self.end:
            return self.weights[key - self.base]
        else:
            return 0

    def polynomial(self):

        z = symbols('z')
        polynomial = 0

        for i in self.indexes:
            polynomial += self.weights[i - self.indexes[0]] * (z ** i)

        return polynomial

In [16]:
def upsampling(filt):

    new_weights = []
    new_indexes = [n for n in 
                    range(2 * filt.indexes[0], 2 * filt.indexes[-1] + 1)]

    for i in range(new_indexes[0], len(new_indexes) + new_indexes[0], 2):
        new_weights.append(filt.weights[int(i/2)])
        new_weights.append(0)
        
    new_weights.pop(-1)

    return Filter(new_weights, new_indexes)

def downsampling(filt):

    new_weights = []
    new_indexes = []

    for index in filt.indexes:
        if index % 2 == 0:
            new_indexes.append(int(index/2))
            new_weights.append(filt.weights[index - filt.indexes[0]])

    return Filter(new_weights, new_indexes)

In [17]:
def numpy_roots(filt):

    filt_vals = filt.weights.copy()
    if filt.indexes[0] == 0:
        filt_vals.reverse()
        return np.roots(filt_vals)
    else:
        vals = []
        i = 0
        while i != filt.indexes[0]:
            vals.append(0)
            i += 1
        for i in filt.weights:
            vals.append(i)
        vals.reverse()
        return np.roots(vals)    
    
def best_match(roots, solve, eps=0.0000001):

    ans=[]
    for sol in solve:
        if sol.is_real:
            isreal = True
        else:
            isreal = False
        for root in roots:
            if isreal:
                if abs(sol - np.real(root)) < eps:
                    ans.append(sol)
            else:
                if abs(re(sol) - np.real(root)) < eps and abs(im(sol) - np.imag(root)) < eps:
                    ans.append([re(sol), im(sol)])

    if len(ans) != len(roots):
        return best_match(roots, solve, eps*10)
    else:
        return ans

def find_roots(filter):
    
    ans = []
    roots = best_match(numpy_roots(filter), solve(filter.polynomial()))
    matches = []
    for i in roots:
        check = 0
        if type(i) == list:
            for j in matches:
                if i[1] == -j[1]:
                    matches.remove(j)
                    check = 1
                    break
            if check == 0:
                matches.append(i)
                ans.append(i)
        else:
            ans.append(i)

    return ans

In [18]:
def rec_comb(n, f0, f1, filters, root_filters, k):

    if n == 0:
        filters.append([Filter([k],[0]) * f0, f1])
    else:
        n -= 1
        rec_comb(n, f0 * root_filters[n - 1], f1, filters, root_filters, k)
        rec_comb(n, f0, f1 * root_filters[n - 1], filters, root_filters, k)

def factorization(f):

    filters = []
    root_filters = []
    root_list = find_roots(f)

    for root in root_list:
        if type(root) == list:
            root_filters.append(Filter([root[0]**2-root[1]**2, -2*root[0], 1],
                                   [0, 1, 2]))
        else:
            root_filters.append(Filter([-root, 1], [0, 1]))

    n = len(root_list)

    f0 = Filter([1], [0])
    f1 = Filter([1], [0])
    k = f.weights[-1]
    rec_comb(n, f0, f1, filters, root_filters, k)

    return filters

In [19]:
def convert_upper_to_lower(h_0, f_0):

    f_1 = Filter(h_0.weights, h_0.indexes)
    h_1 = Filter(f_0.weights, f_0.indexes)

    for i in f_1.indexes:
        if i % 2 == 0:
            f_1.weights[i - f_1.indexes[0]] *= -1

    for i in h_1.indexes:
        if i % 2 == 1:
            h_1.weights[i - h_1.indexes[0]] *= -1

    return (h_1, f_1)

In [91]:
def return_h_0_f_0(letter):

    if letter == "a":
        h_0 = Filter([1], [0])
        f_0 = Filter([-1/16, 0, 9/16, 1, 9/16, 0, -1/16], [0, 1, 2, 3, 4, 5, 6])

    elif letter == "b":
        h_0 = Filter([1/2, 1/2], [0, 1])
        f_0 = Filter([-1/8, 1/8, 1, 1, 1/8, -1/8], [0, 1, 2, 3, 4, 5])

    elif letter == "c":
        h_0 = Filter([1/4, 1/2, 1/4], [0, 1, 2])
        f_0 = Filter([-1/4, 1/2, 3/2, 1/2, -1/4], [0, 1, 2, 3, 4])

    elif letter == "d":
        h_0 = Filter([(2 + sqrt(3))/2, (1 + sqrt(3))/2, -1/2], [0, 1, 2])
        f_0 = Filter([(sqrt(3) - 2)/8, (3 * sqrt(3) - 5)/8, 
                      (3 * sqrt(3) - 3)/8, (sqrt(3) + 1)/8, 1/8], [0, 1, 2, 3, 4])

    elif letter == "e":
        h_0 = Filter([1/8, 3/8, 3/8, 1/8], [0, 1, 2, 3])
        f_0 = Filter([-1/2, 3/2, 3/2, -1/2], [0, 1, 2, 3])

    elif letter == "f":
        h_0 = Filter([(1 + sqrt(3))/(4 * sqrt(2)), (3 + sqrt(3))/(4 * sqrt(2)), 
                     (3 - sqrt(3))/(4 * sqrt(2)), (1 - sqrt(3))/(4 * sqrt(2))], [0, 1, 2, 3])
        f_0 = Filter([(-2 + sqrt(3))/(2 * sqrt(2) * (sqrt(3) - 1)), 
                     (-3 + 2 * sqrt(3))/(2 * sqrt(2) * (sqrt(3) - 1)), 
                     sqrt(3)/(2 * sqrt(2) * (sqrt(3) - 1)), 
                     1/(2 * sqrt(2) * (sqrt(3) - 1))], [0, 1, 2, 3])

    elif letter == "g":
        h_0 = Filter([1/16, 1/4, 3/8, 1/4, 1/16], [0, 1, 2, 3, 4])
        f_0 = Filter([-1, 4, -1], [0, 1, 2])

    return (h_0, f_0)

In [90]:
simplify(np.dot(np.array([(1 + sqrt(3))/(4 * sqrt(2)), (3 + sqrt(3))/(4 * sqrt(2)), 
                     (3 - sqrt(3))/(4 * sqrt(2)), (1 - sqrt(3))/(4 * sqrt(2))]), np.array([(-2 + sqrt(3))/(2 * sqrt(2) * (sqrt(3) - 1)), 
                     (-3 + 2 * sqrt(3))/(2 * sqrt(2) * (sqrt(3) - 1)), 
                     sqrt(3)/(2 * sqrt(2) * (sqrt(3) - 1)), 
                     1/(2 * sqrt(2) * (sqrt(3) - 1))])))

1/4

In [88]:
h_0, f_0 = return_h_0_f_0("f")
h_1, f_1 = convert_upper_to_lower(h_0, f_0)

In [92]:
d_01 = h_0 * downsampling * upsampling * f_0
d_11 = h_1 * downsampling * upsampling * f_1

d = d_01 + d_11
for i in d.weights:
    print(simplify(i))

1/8
0
-3/4
1
-3/8
0


In [26]:
def song_generation(curr, p, h_0, f_0, h_1, f_1, song, sr):
    
    if curr == p:
        return np.array(song)

    d_0 = Filter(song, range(len(song))) * h_0 * downsampling
    d = song_generation(curr + 1, p, h_0, f_0, h_1, f_1, np.array(d_0.weights), sr)
    d_1 = Filter(d, range(len(d))) * upsampling * f_0
    
    return np.array(d_1.weights)
        
def call_rec(letter, p, song, song_name, sr):
    
    h_0, f_0 = return_h_0_f_0(letter)
    h_1, f_1 = convert_upper_to_lower(h_0, f_0)
    
    if not os.path.exists(song_name):
        os.makedirs(song_name)
    
    rec_song = song_generation(0, p, h_0, f_0, h_1, f_1, song, sr)
    wavf.write('{}/{}_{}_{}.wav'.format(song_name, song_name, letter, p), sr, rec_song)

In [27]:
input_song_inst = lr.load('Piano.wav')
song_inst = input_song_inst[0]
sr_inst = input_song_inst[1]
song_name_inst = "Piano"

In [28]:
for letter in ["b", "c", "f"]:
    for num in range(2, 5):
        call_rec(letter, num, song_inst, song_name_inst, sr_inst)

In [75]:
def is_symm(arr):
    for i in range(int(len(arr)/2)):
        if arr[i] != arr[int(len(arr)) - i - 1]:
            return False
    return True

def divide_groups(filt):
    
    ort = []
    lin = []
    pairs = []
    
    #pairs = factorization(filt)
    for letter in ["a", "b", "c", "d", "e", "f", "g"]:
        pairs.append(return_h_0_f_0(letter))
    
    for i in pairs:
        if i == pairs[-2]:
            print(simplify(np.dot(np.array(i[0].weights), 
                            np.array(i[1].weights))))
        if len(i[0].indexes) == len(i[1].indexes) and np.dot(np.array(i[0].weights), 
                                                             np.array(i[1].weights)) == 0:
            ort.append(i)
        if is_symm(i[0].weights) and is_symm(i[1].weights):
            lin.append(i)
    
    return ort, lin

In [76]:
p_0 = Filter([-1/16, 0, 9/16, 1, 9/16, 0, -1/16], range(7))

In [77]:
ort, lin = divide_groups(p_0)

1/4


In [41]:
for i in ort:
    print(i[0])
    print(i[1])
    print()
    
print("============")

for i in lin:
    print(i[0])
    print(i[1])
    print()